<a href="https://colab.research.google.com/github/gaurangsinha/API-Browser/blob/master/AnswersByCommittee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Answers by Committee

Using Large Language Models (LLMs) in a collaborative setting where they need to work together to provide what they collectivelly believe is the best answer to the queries asked of them.



```
               ┌──────────┐                            
               │  PROMPT  │                            
               └────┬─────┘                            
                    │ WHO SHOULD ANSWER THIS?          
                    ▼                                  
             ┌──────────────┐                          
             │ FIND EXPERTS │                          
             └──────┬───────┘                          
                    ▼                                  
┌────────────────────────────────────────┐             
│INITIAL PROMPT FOR RESPONSE USING EXPERT│             
└───────────────────┬────────────────────┘             
                    │                                  
           ─────────▼──────────┐ GET                      
         ─────────────────────┐│ REASONS, ANSWERS      
        ┌────────────────────┐│  AND CONFIDNECE        
        │ GENERATE RESPONSES ││                        
        └───────────┬────────┘                         
                    │                                  
                    ▼                                  
          ┌────────────────────┐ GET                      
┌─────────► CRITIQUE RESPONSES │ CRITIQUES & CONFIDENCE
│         └─────────┬──────────┘ FROM OTHER MEMBERS                      
│                   ▼                                  
│         ┌────────────────────┐ GET                      
│         │  REFINE RESPONSES  │ REASONS, ANSWERS      
│         └─────────┬──────────┘ AND CONFIDENCE        
│BELOW THRESHOLD    ▼                                  
│         ┌────────────────────┐                       
└─────────┼   CHECK THRESHOLD  │                       
          └─────────┬──────────┘                       
                    │ MEETS THRESHOLD                  
                    ▼                                  
             DISPLAY RESPONSE                          

```



In [ ]:
# @title Initialize {"display-mode":"form"}
!pip install llm
!pip install llm-claude-3
!pip install llm-gemini

from google.colab import userdata
import logging

logger = logging.getLogger('my_logger')

logging.basicConfig(
    format='%(asctime)s - %(message)s',
    level=logging.DEBUG)
logger.warning('Anaswers by Committee has been initialized!')

In [ ]:
from __future__ import annotations
from typing import List, Callable
from google.colab import userdata
import llm
import re

EXPERTS_PROMPT = \
"""Based on the prompt enclosed in the <prompt> tags, provide an expert that would be best suited to answer the query.
Ensure you enclose your response provided within the <expert> tag. Ensure you provide only the role within the tag, any other information may be provided outside of the tag.

<prompt>
{PROMPT}
</prompt>
"""

INITIAL_PROMPT = \
"""You are a well respected {EXPERT}. You work closely with your colleagues on a committee of experts.
The committee considers challenges and limitations while exploring multiple perspectives.
Carefully consider the instructions below on how to interact on this committee:
- The committee works in an structure manner where each member provides a response in a structure defined below.
- The query will be enclosed witin a <prompt> tag.
- Start by providing reasoning for your answer and enclosing it within the <reason> tag.
- Provide the answer by enclosing it within the <answer> tag.
- Finally provide a confidence number between 0.0 to 1.0 where 0 means no confidence and 1 means 100% confidence within the <confidence> tag.

Your response should look like this

<reason>
[your reasoning for the answer provided]
</reason>

<answer>
[your answer]
</answer>

<confidence>
[your confidence number]
</confidence>
"""

CRITIQUE_PROMPT = """
As a well respected {EXPERT} and member of the expert committee your role is to provide clear, concise and actionable critique on the data provided.
Carefully consider the instructions below on how to read the data provided and respose to be given.
You will be provided with the data in the following structure
- The original prompt which was to be answered will be enclosed within the <prompt> tag.
- The reasoning for the answer given will be enclosed within the <reason> tag.
- The answer given will be enclosed within the <answer> tag.

For the response
- Consider the reason and answer provided in context of the original prompt
- Your critiques should enable the author to improve the answer, respond using the following structure
- Start by providing critique enclosed within the <critique> tag.
- Finally provide a confidence number between 0.0 to 1.0 where 0 means no confidence and 1 means 100% confidence within the <confidence> tag.
"""

REFINEMENT_PROMPT = \
"""You are a well respected {EXPERT}. You work closely with your colleagues on a committee of experts.
The committee considers challenges and limitations while exploring multiple perspectives.
Carefully consider the instructions below on how to interact on this committee:
- The committee works in an structure manner where each member provides a response in a structure defined below.
- The query will be enclosed witin a <prompt> tag.
- Your previous answer will be enclosed within the <answer> tag.
- Critiques from various members on the committee on your answer will be enclosed in the <critique> tag.
- Carefully understand the context provided above and start your response using the structure defined below.
- Start by providing reasoning for your answer and enclosing it within the <reason> tag.
- Provide the answer by enclosing it within the <answer> tag.
- Finally provide a confidence number between 0.0 to 1.0 where 0 means no confidence and 1 means 100% confidence within the <confidence> tag.

Your response should look like this

<reason>
[your reasoning for the answer provided]
</reason>

<answer>
[your answer]
</answer>

<confidence>
[your confidence number]
</confidence>
"""

def extract_within_tag(text:str, tag:str) -> str:
  return re.findall(r"<" + tag + r">(.*?)</" + tag + r">", text, re.DOTALL)[0].strip()

class Committee:
  def __init__(self, members: List[CommitteeMember], confidence_threshold:float=0.8, regeneration_count:int=3) -> None:
    logging.warning(f"Committee __init__  {len(members)} members")
    self.members = members
    self.prompt = ""
    self.confidence_threshold = confidence_threshold
    self.regeneration_count = regeneration_count

  def set_prompt(self, prompt: str):
    logging.warning(f"Committee set_prompt {prompt}")
    self.prompt = prompt
    for member in self.members:
      member.set_prompt(prompt=self.prompt)

  def start_discussion(self) -> str:
    logging.warning(f"Committee start_discussion")
    # generate first set of responses
    for member in self.members:
      member.generate_response()

    logging.warning(f"Committee start_discussion fetch feedback")
    # get feedback
    for id_i, member in enumerate(self.members):
      for id_j, other_member in enumerate(self.members):
        if id_i == id_j:
          continue # dont generate feedback from the same model that generated the answer
        member.generate_critique(for_response=other_member.latest_response)

    # regenerate answers if required
    logging.warning(f"Committee start_discussion check feedback threshold={self.confidence_threshold}")
    for member in self.members:
      logging.warning(f"Committee start_discussion {member.name} confidence={member.latest_response.confidence} average_critic_condidence={member.average_critic_condidence}")
      if member.average_critic_condidence < self.confidence_threshold and len(member.responses) < self.regeneration_count:
        logging.warning(f"Committee start_discussion {member.name} regenerating answer")
        member.generate_response()

    # summarize all individual answers and return to user
    logging.warning(f"Committee start_discussion generating summary")
    answer_list = [m.latest_response.answer for m in self.members]
    answer_list_str = "\n".join(map(lambda a: f"<text>{a}</text>", answer_list))
    summaries = []
    for member in self.members:
      summary_raw = member.model_prompt_func(
          prompt=answer_list_str,
          system_prompt=f"As a qualified {member.expert} please summarize the following set of texts enclosed in the <text> tags. Ensure your summary is accurate represents the original material.")
      logging.warning(f"Committee start_discussion summary {member.name} {summary_raw}")
      summaries.append(summary_raw)

    logging.warning(f"Committee start_discussion pick best summary")
    for member in self.members:
      print(member.model_prompt_func(prompt="\n-----\n".join(summaries),
                               system_prompt=f"Given the following set of text separated by ----- pick the one that best communicates the answer for {self.prompt}. Do not provide any additional information, just print out the answer you prefer."))


class CommitteeMember:
  def __init__(self, name: str, model_prompt_func: Callable[[str, str], str]):
    logging.warning(f"CommitteeMember __init__ {name}")
    self.name = name
    self.model_prompt_func = model_prompt_func

    self.prompt = ""

    self._expert_response = None
    self.expert = ""

    self._initial_system_prompt = ""
    self._critique_system_prompt = ""
    self._refinement_system_prompt = ""

    self.responses = []
    self.critique_given = []


  def set_prompt(self, prompt: str):
    logging.warning(f"CommitteeMember {self.name} set_prompt {prompt}")
    self.prompt = prompt
    self._find_expert_and_set_system_prompts(self.prompt)

  def _extract_expert(self, text: str) -> str:
    logging.warning(f"CommitteeMember {self.name} _extract_expert {text}")
    return extract_within_tag(text, "expert")

  def _find_expert_and_set_system_prompts(self, prompt: str):
    global EXPERTS_PROMPT, INITIAL_PROMPT, CRITIQUE_PROMPT, REFINEMENT_PROMPT
    logging.warning(f"CommitteeMember {self.name} _find_expert_and_set_system_prompts {prompt}")
    self._expert_response = self.model_prompt_func(
        prompt=EXPERTS_PROMPT.format(PROMPT=prompt), system_prompt=None)
    self.expert = self._extract_expert(self._expert_response)
    self._initial_system_prompt = INITIAL_PROMPT.format(EXPERT=self.expert)
    self._critique_system_prompt = CRITIQUE_PROMPT.format(EXPERT=self.expert)
    self._refinement_system_prompt = REFINEMENT_PROMPT.format(EXPERT=self.expert)

  def get_recevied_critique_summary(self) -> str:
    logging.warning(f"CommitteeMember {self.name} get_recevied_critique_summary")
    return "\n".join(map(lambda c: c.critique, self.critique_received))

  def generate_response(self):
    prompt = f"<prompt>{self.prompt}</prompt>"
    if(len(self.responses) > 0):
      prompt = f"<prompt>{self.prompt}</prompt>\n<answer>{self.latest_response.answer}</answer>\n<critique>{self.get_recevied_critique_summary()}</critique>"
    logging.warning(f"CommitteeMember {self.name} generate_response {prompt}")
    raw_response = self.model_prompt_func(prompt=prompt,
                                      system_prompt=self._refinement_system_prompt)
    self.responses.append(CommitteeMemberResponse(member=self,
                                              prompt=self.prompt,
                                              raw_response=raw_response))

  def generate_critique(self, for_response: CommitteeMemberResponse):
    prompt=f"<prompt>{for_response.prompt}</prompt>\n<reason>{for_response.reason}</reason>\n<answer>{for_response.answer}</answer>"
    logging.warning(f"CommitteeMember {self.name} generate_critique {prompt}")
    critique_response = self.model_prompt_func(
        prompt=prompt,
        system_prompt=self._critique_system_prompt)
    critique = CommitteeMemberCritique(critic=self,
                                       response=for_response,
                                       raw_critique=critique_response)
    self.critique_given.append(critique)
    for_response.add_critique(critique)

  @property
  def latest_response(self) -> CommitteeMemberResponse:
    return self.responses[-1] if len(self.responses) > 0 else None

  @property
  def critique_received(self) -> List[CommitteeMemberCritique]:
    return [critique for response in self.responses for critique in response.critiques]

  @property
  def average_critic_condidence(self) -> float:
    return sum(map(lambda c: c.confidence, self.critique_received)) / len(self.critique_received)

  def __str__(self):
    return f"Name: {self.name}\nExpert: {self.expert}"

class CommitteeMemberCritique:
  def __init__(self, critic: CommitteeMember, response: CommitteeMemberResponse, raw_critique: str):
    logging.warning(f"CommitteeMemberCritique __init__ {critic.name} {response.member.name}")
    self.critic = critic
    self.response = response
    self._raw_critique = raw_critique
    self.critique = ""
    self.confidence = 0.0
    self._parse_critique(self._raw_critique)

  def _parse_critique(self, text: str):
    logging.warning(f"CommitteeMemberCritique _parse_critique {text}")
    self.critique = extract_within_tag(text, "critique")
    self.confidence = float(extract_within_tag(text, "confidence"))
    logging.warning(f"CommitteeMemberCritique _parse_critique confidence={self.confidence}")

  def __str__(self):
    return f"Critique By\n{self.critic}\nCritique For\n{self.response.member}\nCritique: {self.critique}\nConfidence: {self.confidence}"

class CommitteeMemberResponse:
  def __init__(self, member: CommitteeMember, prompt: str, raw_response: str):
    logging.warning(f"CommitteeMemberResponse __init__ {member.name}")
    self._raw_response = raw_response
    self.member = member
    self.prompt = prompt
    self.answer = ""
    self.reason = ""
    self.confidence = 0.0
    self.critiques : CommitteeMemberCritique = []
    self._parse_response(self._raw_response)

  def _parse_response(self, text: str):
    logging.warning(f"CommitteeMemberResponse _parse_response {text}")
    self.reason = extract_within_tag(text, "reason")
    self.answer = extract_within_tag(text, "answer")
    self.confidence = float(extract_within_tag(text, "confidence"))
    logging.warning(f"CommitteeMemberResponse _parse_response confidence={self.confidence}")

  def add_critique(self, critique: CommitteeMemberCritique):
    self.critiques.append(critique)

  @property
  def average_critic_condidence(self) -> float:
    return sum(map(lambda c: c.confidence, self.critiques)) / len(self.critiques)

  def __str__(self):
    return f"{self.member}\nReason: {self.reason}\nAnswer: {self.answer}\nConfidence: {self.confidence}"


PROMPT = "What would be an ideal voting system to put in place for a new country being formed today?" #@param {"type":"string"}

def create_model_prompt_func(name: str, key: str) -> Callable[[str, str], str]:
  logging.warning(f"Creating committee member {name} prompt function using key {key}")
  def prompt_func(prompt: str, system_prompt: str = None) -> str:
    logging.warning(f"prompt_func {name} with {prompt} system_prompt {system_prompt}")
    model = llm.get_model(name=name)
    model.key = userdata.get(key)
    response = model.prompt(prompt=prompt, system=system_prompt, stream=False)
    return response.text() if response is not None else ""
  return prompt_func

logging.warning(f"Using prompt `{PROMPT}` for committee delibrations")
logging.warning(f"Creating committee members")

members = [
    CommitteeMember(
        name="gpt-4-1106-preview",
        model_prompt_func=create_model_prompt_func("gpt-4-1106-preview", "OPENAI_API_KEY")),
    # CommitteeMember(
    #     name="gemini-exp-1206",
    #     model_prompt_func=create_model_prompt_func("gemini-exp-1206", "GOOGLE_API_KEY")),
    CommitteeMember(
        name="claude-3-sonnet-20240229",
        model_prompt_func=create_model_prompt_func("claude-3-sonnet-20240229", "CLAUDE_API_KEY"))
]

committee = Committee(members=members)

committee.set_prompt(prompt=PROMPT)

committee.start_discussion()


# resp_gpt4 = gpt4.prompt(prompt=EXPERTS_PROMPT, stream=False)
# resp_gemini = gemini.prompt(prompt=EXPERTS_PROMPT, stream=False)
# resp_claude = claude.prompt(prompt=EXPERTS_PROMPT, stream=False)


# resp_gpt4 = gpt4.prompt(prompt=f"<prompt>{PROMPT}</prompt>", system=SYSTEM_PROMPT, stream=False)
# print(resp_gpt4.text())

# resp_gemini = gemini.prompt(prompt=f"<prompt>{PROMPT}</prompt>", system=SYSTEM_PROMPT, stream=False)
# print(resp_gemini.text())

# resp_claude = claude.prompt(prompt=f"<prompt>{PROMPT}</prompt>", system=SYSTEM_PROMPT, stream=False)
# print(resp_claude.text())


Ensure you enclose your response provided within the <expert> tag. Ensure you provide only the role within the tag, any other information may be provided outside of the tag.

<prompt>
What would be an ideal voting system to put in place for a new country being formed today?
</prompt>
 system_prompt None
Ensure you enclose your response provided within the <expert> tag. Ensure you provide only the role within the tag, any other information may be provided outside of the tag.

<prompt>
What would be an ideal voting system to put in place for a new country being formed today?
</prompt>
 system_prompt None

A political scientist with expertise in electoral systems, voting theory, and comparative politics would be best suited to answer this query. They would have a deep understanding of different voting models, their advantages and disadvantages, and how they might be applied in the context of forming a new country. Additionally, their knowledge of political structures and democratic proces

The Mixed-Member Proportional (MMP) voting system is recommended for a new country as it offers a combination of proportional representation and single-member districts, leading to a legislature that reflects overall voter preferences while also maintaining local representation. This balance allows for diverse political viewpoints to be proportionately represented, fostering inclusive governance, accountability, and political stability. MMP discourages single-party dominance and often necessitates coalition governments, which can result in more collaborative and representative governance.
The Mixed-Member Proportional (MMP) voting system is recommended for a new country as it offers a combination of proportional representation and single-member districts, leading to a legislature that reflects overall voter preferences while also maintaining local representation. This balance allows for diverse political viewpoints to be proportionately represented, fostering inclusive governance, acco